In [1]:
import preprocess as pp
import configs
import persister
import relation
import lda
import evaluate
import embedding
from utils import *

from collections import Counter
from pprint import pprint
from functools import reduce

import numpy as np

not use stanford CoreNLP client!


In [11]:
l = 7
t = 50
m = "c_v"
i = 200
min_df = 1
size = 100
parse = persister.read_parse()
_input = persister.read_input(configs.NEWSINPUT)
model_name = configs.NEWSMODEL.format(l, t, m, i, min_df)
terms, doc_topic, topic_word = persister.read_lda(configs.NEWSLDA.format(model_name))
model = persister.load_model(model_name)
vec = persister.load_model(configs.NEWSVEC.format(min_df))
w2vmodel = persister.load_wv(configs.NEWSWV.format(size))
tf = vec.fit_transform(_input)
top_terms, top_docs = lda.get_topics(topic_word, terms, doc_topic, 10)
word_values = []
for i in top_terms:
    tmp = []
    for j in i:
        tmp.append((terms[j[0]], j[1]))
#         tmp.append(terms[j[0]])
    word_values.append(tmp)
df_top_words, df_top_docs = lda.pd_topics_vis(word_values, top_docs)

In [3]:
extended = relation.extend_lda_results(parse, _input, top_terms, top_docs, terms, "originalText", top_n=10, score_method="tf")

In [4]:
train_data, train_label, test_data, test_label = evaluate.split_data_set(doc_topic)
print(evaluate.svm(train_data, train_label, test_data, test_label))

0.8075


In [5]:
words, weights = evaluate.normalize_top_terms(top_terms, terms)
hybrid_data = evaluate.get_hybrid_feature(_input, w2vmodel, words, weights)
train_data, train_label, test_data, test_label = evaluate.split_data_set(hybrid_data)
print(evaluate.svm(train_data, train_label, test_data, test_label))

0.2875


In [7]:
extended_words,_ = evaluate.filter_words(extended, topic_word, vec, top_terms, top_docs, _input, terms)
words, weights = evaluate.convert_extended_words2weights(extended_words, topic_word, vec)
hybrid_data = evaluate.get_hybrid_feature(_input, w2vmodel, words, weights)
train_data, train_label, test_data, test_label = evaluate.split_data_set(hybrid_data)
evaluate.svm(train_data, train_label, test_data, test_label)

0.255

In [8]:
w2vmodel.wv["assess"]

array([-1.41043970e-02,  1.89622361e-02,  6.31992333e-03,  1.00738704e-02,
       -1.55927306e-02, -2.17699143e-03,  1.77599827e-03,  5.73018054e-03,
       -1.65819712e-02,  1.28262043e-02, -7.14619877e-03,  4.60338872e-03,
        1.40667625e-03, -1.20086772e-02, -1.13446955e-02, -2.25290004e-03,
        7.10217841e-03,  8.95414315e-03,  9.60995350e-03,  1.06498161e-02,
       -1.52899083e-04, -1.35321133e-02,  8.45884133e-05,  2.29108240e-03,
        3.33277276e-03,  4.99730790e-03,  3.25394957e-03,  5.74186258e-03,
        1.59190316e-02,  6.05825335e-03, -7.22621195e-03, -7.98995513e-03,
        1.20820878e-02, -2.56545283e-03, -1.21602369e-02,  3.90509202e-04,
       -3.53875756e-03, -3.07417149e-03, -7.16265989e-03,  5.15754009e-03,
       -2.44596098e-02, -4.66752099e-03,  6.43143337e-03, -1.40998475e-02,
       -8.85499269e-03, -6.16792915e-03, -1.38281647e-03,  3.11542791e-03,
        4.02312446e-03, -2.40615662e-03,  8.75603873e-03, -1.02104256e-02,
       -5.55870123e-03, -

In [9]:
words, weights = evaluate.normalize_top_terms(top_terms, terms)
for idx,w in enumerate(words):
    s = evaluate.get_topic_vec(w,weights[idx],w2vmodel)
    if not s.any():
        print(w)
        v = []
        for ww in w:
            if ww in w2vmodel.wv:
                v.append(True)
            else:
                v.append(False)
        print(v)
        print([e[0] for e in weights[idx]])